In [0]:
import os
import sys
import asyncio
from openai import AsyncAzureOpenAI

In [0]:
def create_input_file(input_file_name):
    function_code = """
                    def absolute_square(num1, num2):
                        result = abs(num1 - num2)
                        result *= result
                        return result
                    """

    with open(input_file_name, 'w') as file:
        file.write(function_code)

    print(f"Created and wrote to file: {input_file_name}")

In [0]:
def read_file(file_path):
    """Reads the content of the specified file."""
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except Exception as e:
        print(f"Error reading file: {e}")
        sys.exit(1)

In [0]:
def save_file(content):
    """Saves the generated content to a new file."""
    try:
        # Creating an empty text file named output.txt
        file_name = 'output.txt'

        with open(file_name, 'w') as file:
            pass

        print(f"Created file: {file_name}")
    except Exception as e:
        print(f"Error saving file: {e}")
        sys.exit(1)

In [0]:
async def generate_commented_code(original_code, prompt, model, client):
    """Generates commented code using Azure OpenAI."""
    prompt = f"{prompt}\n\n`{original_code}`"
    # Provide a basic user message, and use the prompt content as the user message
    system_message = "You are a helpful AI assistant that helps programmers write code."
    user_message = prompt

    # Format and send the request to the model
    messages =[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
    try:
    # Call the Azure OpenAI model
      response = await client.chat.completions.create(
          model=model,
          messages=messages,
          temperature=0.5,
          max_tokens=1000
      )

      return response.choices[0].message.content.strip() 
    except Exception as e:
        print(f"Error generating commented code: {e}")
        sys.exit(1)

In [0]:
# Set to True to print the full response from OpenAI for each call
printFullResponse = False

async def main():
  try:
        # Get configuration settings
        # Configuration settings
        azure_oai_endpoint =""
        azure_oai_key =""
        azure_oai_deployment =""

        # Configure the Azure OpenAI client
        client = AsyncAzureOpenAI(
            azure_endpoint = azure_oai_endpoint,
            api_key=azure_oai_key,
            api_version="2024-02-15-preview"
        )

        user_input = input('\nUsage: <input_file.py>')
        input_file = sys.argv[0]
        create_input_file(input_file)

        # Read the input file
        original_code = open(file=input_file, encoding="utf8").read()
        print(f"Original code:\n {original_code} ")

        # Define your prompt for the OpenAI model
        prompt = (
            "Take the following Python code and rewrite it with detailed comments that explains what each part of the code is doing."
        )

        # Generate commented code
        commented_code = await generate_commented_code(original_code, prompt=prompt, model=azure_oai_deployment, client=client)

        print(f"{commented_code}")
        # Save the output
        save_file(commented_code)
        results_file = open(file="output.txt", mode="w", encoding="utf8")
        results_file.write(commented_code)
        print(f"Final commented code writtent to output.txt")

  except Exception as ex:
        print(ex)

In [0]:
# Run the main function in the event loop
await main()